# Chargement des librairies

In [2]:
library(sparklyr)
library(tidyverse, warn.conflicts = FALSE)
library(lubridate)
library(tableone)
library(fuzzyjoin)
library(fastDummies)


Attaching package: ‘sparklyr’


The following object is masked from ‘package:SparkR’:

    collect


Warning message:
“Failed to locate timezone database”
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.4.3     ✔ purrr   1.0.2
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()      masks SparkR::arrange()
✖ dplyr::between()      masks SparkR::between()
✖ dplyr::coalesce()     masks SparkR::coalesce()
✖ dplyr::collect()      masks sparklyr::collect(), SparkR::collect()
✖ dplyr::contains()     masks tidyr::contains(), SparkR::contains()
✖ dplyr::count()        masks SparkR::count()
✖ dplyr::cume_dist()    masks SparkR::cume_dist()
✖ dplyr::dense_rank()   masks SparkR::dense_rank()
✖ dplyr::desc()         masks SparkR::desc()
✖ dplyr::distinct()     masks SparkR::distinct()
✖ dplyr::expl

# Connexion à Spark

In [3]:
config <- spark_config()
sc <- spark_connect(master = "yarn-cluster", config = config, version = '2.4.3')

# Sélection du schéma FA

In [4]:
tbl_change_db(sc, 'cse_200007_20210519')

# Définition des variables

## UFR de réanimation

In [5]:
liste_reas_med <- c('UFR:028091',  'UFR:014025', 'UFR:095013', 'UFR:011440', 'UFR:010030',
                    'UFR:010502', 'UFR:021034', 'UFR:0101', 'UFR:026151', 'UFR:053011', 'UFR:047057',
                    'UFR:066067', 'UFR:066228', 'UFR:068033', 'UFR:073290', 'UFR:076023', 'UFR:087260')

liste_reas_chir <- c('UFR:095073', 'UFR:095062', 'UFR:010025', 'UFR:010260', 'UFR:005269', 'UFR:005389', 'UFR:021262', 'UFR:0613', 
                     'UFR:026513', 'UFR:026514', 'UFR:047242', 'UFR:061726', 'UFR:066020', 'UFR:066120', 'UFR:066420', 'UFR:066520', 
                     'UFR:066620', 'UFR:073260', 'UFR:073261')

liste_reas_enlevees <- c('UFR:028402', 'UFR:010302', 'UFR:053036', 'UFR:026511', 'UFR:066320', 'UFR:0107')

liste_reas <- c(liste_reas_med, liste_reas_chir)

liste_reas_non_triees <- c(liste_reas, liste_reas_enlevees)

## Liste des UFR laissées de côté après réanalyse de la liste

In [6]:
### Soins continus : c('UFR:028402', 'UFR:010302', 'UFR:053036')
### Chirurgie cardiaque : c('UFR:026511', 'UFR:066320')

## Examens de laboratoire

### Codes des examens de laboratoire d'hémostase

In [7]:
#### AntiXa-HNF : LAB:A0350, LAB:G1986, LAB:E3109, 
#### AntiXa-HBPM : LAB:A0018, LAB:E3108
#### Danaparoïde sodique : LAB:A0019, LAB:A2662, LAB:H6291, LAB:D9103
#### TCA : LAB:A0265 (Patient), LAB:A0266 (Témoin), LAB:A1792 (Ratio)
#### INR : LAB:A0269, LAB:F5455
#### AOD : LAB:E5350 (Rivaroxaban), LAB:F8012 (Apixaban), LAB:H3248 (Edoxaban)

lab_antiXa_hnf <- c ('LAB:A0350', 'LAB:G1986', 'LAB:E3109')
lab_antiXa_hbpm <- c ('LAB:A0018', 'LAB:E3108')
lab_tca_ratio <- c ('LAB:A1792')
lab_aod <- c('LAB:E5350', 'LAB:F8012', 'LAB:H3248')

lab_tous_tests <- c(lab_antiXa_hnf,lab_antiXa_hbpm,lab_tca_ratio,lab_aod)
lab_tous_tests_sans_tca <- c(lab_antiXa_hnf,lab_antiXa_hbpm,lab_aod)

### Codes des autres examens de laboratoire pertinents

In [8]:
## Codes de l'Hb - NB B1946 et B1945 sont l'Hb des GdS veineux et artériels respectivement, donc 
## discuter la variabilité des valeurs entre GdS et NFS.
lab_hb <- c('LAB:A0163', 'LAB:C1549', 'LAB:A9882', 'LAB:C8745', 'LAB:B1946', 'LAB:B1945')

## Correspondance LOINC : 22664-7
lab_uree <- c('LAB:A0286', 'LAB:G3350')

# Correspondance LOINC : 3255-7
lab_fib <- c('LAB:A0126', 'LAB:A1114', 'LAB:A2667')

## Correspondance LOINC : 777-3
## Correspondance LOINC : 778-1
lab_plq <- c('LAB:A0230', 'LAB:A2538', 'LAB:B9896', 'LAB:G5746', 'LAB:G7727', 'LAB:G7728', 'LAB:G7833', 'LAB:A1598', 'LAB:A2539')

## Correspondance LOINC : 48066-5
lab_ddim <- c('LAB:B4199', 'LAB:C0474', 'LAB:C7882', 'LAB:F5402')

## Correspondance LOINC : 14682-9
lab_creat <- c('LAB:F9409','LAB:C0697','LAB:F9410','LAB:F2621','LAB:G7834','LAB:A7813','LAB:H4038','LAB:A0094')

lab_tous_autres <- c(lab_hb, lab_uree, lab_fib, lab_plq, lab_ddim, lab_creat)

## Variables des actes CCAM

In [9]:
ccam_iot <- 'CCAM:GELD004'
ccam_vm <- c('CCAM:GLLD015', 'CCAM:GLLD004', 'CCAM:GLLD008')
ccam_vasopresseur <- c('CCAM:EQLF003', 'CCAM:EQLF001')
ccam_eer <- c('CCAM:JVJF002', 'CCAM:JVJF005', 'CCAM:JVJB002')
ccam_ecmo <- 'CCAM:EQQP004'
ccam_transfusion <- c('CCAM:FELF011', 'CCAM:FELF004')
ccam_coagulopathie <- 'CCAM:FELF003'

## Variables des codages diagnostics CIM10

### Codes ischémiques

In [10]:
#### I63 : AIC
#### I74 : Embolie ou thrombose artérielle
#### I513 : Thrombose intra-cardiaque
#### K55 : Troubles vasculaires aigus de l'intestin
#### N28 : Infarctus rénal
#### D735 : Infarctus splénique

ischemie <- tibble(code = c('CIM10:I63', 'CIM10:I74', 'CIM10:I513', 'CIM10:K550', 'CIM10:N280', 'CIM10:D735'),
                   type = c('cerebrale', 'vasculaire', 'cardiaque', 'intestinal', 'renal', 'splenique'),
                   binaire_ischemie = c(1,1,1,1,1,1))

diag_biais_anticoag <- tibble(code = c('CIM10:I74', 'CIM10:I513', 'CIM10:K550', 'CIM10:I26', 'CIM10:Z952', 'CIM10:I80', 'CIM10:I81', 'CIM10:I82', 'CIM10:D685', 'CIM10:D686'),
                        diag = c('b_thrombose_arterielle', 'b_thrombose_cardiaque', 'b_ischemie_mesenterique', 'b_embolie_pulmonaire', 'b_valve_cardiaque', 'b_thrombose_veineuse', 'b_thrombose_veineuse', 'b_thrombose_veineuse', 'b_thrombophilie', 'b_thrombophilie'),
                        binaire_biais_anticoag = c(1,1,1,1,1,1,1,1,1,1))

diag_biais_anticoag_alter <- tibble(code = c('CIM10:Z952', 'CIM10:D685', 'CIM10:D686'),
                        diag = c('b_valve_cardiaque', 'b_thrombophilie', 'b_thrombophilie'),
                        binaire_biais_anticoag = c(1,1,1))

### Codes hémorragiques 

In [11]:
#### I61 : Hémorragie intracérébrale
#### I62 : Autre hémorragies intracrâniennes
#### R04 : Hémorragie des voies respiratoires
#### T7952 : 
#### T810 : Hémorragie compliquant un acte à visée diagnostique ou thérapeutique
#### M250 : Hémarthrose
#### R58 : Hémorragie, non classée ailleurs
#### N328 : Autres affections précisées de la vessie (un peu large, ça ?)
#### R31 : Hématurie
#### K922 : Hémorragie gastro-intestinale
#### K920 : Hématémèse
#### K661 : Hémopéritoine
#### K250 : Ulcère aigu de l'estomac
#### K260 : Ulcère aigu du duodénum
#### K270 : Ulcère digestif de siège non précisé avec hémorragie
#### K280 : Ulcère gastro-jéjunal aigu avec hémorragie
#### K2X4 : Ulcère [de localisation correspondant au X] chronique ou non précisé, avec hémorragie

hemorragie <- tibble(code = c('CIM10:I61', 'CIM10:I62', 'CIM10:R04', 'CIM10:T7952', 'CIM10:T810',
                      'CIM10:M250', 'CIM10:R58', 'CIM10:N328', 'CIM10:R31', 'CIM10:K922',
                      'CIM10:K920', 'CIM10:K661', 'CIM10:K260', 'CIM10:K250', 'CIM10:K280',
                      'CIM10:K270', 'CIM10:K264', 'CIM10:K254', 'CIM10:K284', 'CIM10:K274'),
                     type = c('cerebrale', 'cerebrale', 'respiratoire', 'autre', 'iatrogene',
                              'articulaire', 'autre', 'urinaire', 'urinaire', 'digestif', 'digestif',
                              'digestif', 'digestif', 'digestif', 'digestif', 'digestif', 'digestif',
                              'digestif', 'digestif', 'digestif'),
                     binaire_hemorragie = c(1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1))

In [12]:
#### E11/E10 : Diabète
#### I10 : Hypertension artérielle
#### E66 : Obésité
#### J44/J42 : BPCO
#### I50 : Insuffisance cardiaque chronique 
#### N18 : Maladie rénale chronique

antecedents <- tibble(code = c('CIM10:E11', 'CIM10:E10', 'CIM10:I10', 'CIM10:E66', 'CIM10:J44','CIM10:J42', 'CIM10:I50', 'CIM10:N18'),
                     type = c('diabete', 'diabete', 'hta', 'obesite', 'bpco', 'bpco','ins_cardiaque', 'ins_renale'),
                     binaire_atcd = c(1,1,1,1,1,1,1,1))

### Codes autres
#### Etats de choc

In [13]:
etat_choc <- tibble(code=c('CIM10:R572', 'CIM10:R571', 'CIM10:R570', 'CIM10:R578', 'CIM10:R579'),
             type_choc=c('septique', 'hypovolemique', 'cardiogenique', 'autre', 'autre'),
             binaire_choc=c(1,1,1,1,1))

#### Sepsis

In [14]:
sepsis <- 'CIM10:R650'

#### Neurochirurgie

In [15]:
# Code des actes chirurgicaux 

## Neurochirurgie encéphalique et médullaire

nc <- tibble(code = c('CCAM:ACFA', 'CCAM:ABJA', 'CCAM:ABCC', 'CCAM:ABCA', 'CCAM:ABMA', 'CCAM:AAJA', 'CCAM:AAJH',
                      'CCAM:ABFA', 'CCAM:ABSA', 'CCAM:AAFA', 'CCAM:ABFC', 'CCAM:ABJC'),
             type = c('cerebrale', 'cerebrale', 'cerebrale', 'cerebrale', 'cerebrale', 'cerebrale', 'cerebrale', 
                      'cerebrale', 'cerebrale', 'cerebrale', 'cerebrale', 'cerebrale'))

# Créations des tables

## Fibrillation atriale (FA)

### FA de novo

In [16]:
fa_de_novo <- sc %>% tbl('i2b2_observation_cim10') %>%
    select(location_cd, concept_cd, encounter_num, patient_num, start_date) %>%
    mutate(concept_cd_abr = str_sub(concept_cd, 1, 9)) %>%
    filter(concept_cd_abr == "CIM10:I48") %>%
    collect() %>%
    group_by(patient_num) %>%
    slice_min(start_date) %>%
    mutate(fadenovo = ifelse(location_cd %in% liste_reas, 1, 0)) %>%
    select(-c(concept_cd, concept_cd_abr, start_date)) %>%
    filter(location_cd %in% liste_reas) %>%
    distinct()

In [17]:
fa_de_novo <- copy_to(sc, fa_de_novo, overwrite = TRUE)

### Codages de FA

In [18]:
codages_fa <- sc %>% tbl('i2b2_observation_cim10') %>%
    select(location_cd, concept_cd, encounter_num, patient_num, start_date) %>%
    mutate(concept_cd_abr = str_sub(concept_cd, 1, 9)) %>%
    filter(concept_cd_abr == "CIM10:I48") %>%
    select(-c(concept_cd, start_date, concept_cd_abr))

### Diagnostics interférant avec la décision d'anticoagulation
#### Biais thrombotiques
Quels que soient leurs qualification diagnostique (DP ou DAS) car une embolie pulmonaire diagnostiquée au cours du séjour sera une indication formelle. Est-ce vraiment une bonne idée si on considère par exemple la dialyse qui est également une indication d'anticoagulation mais qui n'est pas exclue.

In [19]:
cim10_biais_thrombose <-  sc %>% tbl('i2b2_observation_cim10') %>%
    select(encounter_num, patient_num, concept_cd, start_date, instance_num, tval_char, location_cd) %>%
    filter(location_cd %in% liste_reas) %>%
    collect() %>%
    filter(str_detect(concept_cd, paste(diag_biais_anticoag$code, collapse = "|")))

#### Biais hémorragiques
Où ne sont gardées que les causes d'admission en réanimation (biais ?) car une hémorragie en cours de réanimation n'interrompt pas toute indication d'anticoagulation

In [20]:
cim10_biais_hemorragie <- sc %>% tbl("i2b2_observation_cim10") %>% 
    select(encounter_num, patient_num, concept_cd, start_date, instance_num, tval_char, location_cd) %>%
    filter(location_cd %in% liste_reas) %>%
    collect() %>%
    filter(str_detect(concept_cd, paste(hemorragie$code, collapse = "|"))) %>%
    filter(tval_char == 'DP')

#### Visites avec un acte neurochirurgical

In [21]:
visites_avec_nc <- sc %>% tbl('i2b2_observation_ccam') %>%
    mutate(concept_cd_abr = str_sub(concept_cd, 1, 9)) %>%
    filter(concept_cd_abr %in% !!nc$code) %>%
    as_tibble() %>%
    group_by(encounter_num) %>%
    count()

## Séjours

 - A noter que le semi-join permet d'enlever l'ensemble des séjours qui n'ont pas d'examens de laboratoire

In [22]:
## Table de synthèse avec les données des patients, des séjours (une ligne par séjour, donc plusieurs lignes)
## possibles par patient unique, de la biologie (valeurs minimales et maximales de : Hb, Urée, TP, TCA,
## anti-XA, Plq)

## Séjours de réanimation filtrés depuis la table UFR - sejours_rea_48_heures

sejours_rea_48_heures <- sc %>% tbl('i2b2_observation_ufr') %>% 
    filter(location_cd %in% liste_reas) %>%
    select(encounter_num, patient_num, location_cd, start_date, end_date, instance_num) %>%

### Fusion avec les tables  Patient et Visite pour obtenir l'âge de chaque patient ainsi que son statut vital au moment de chaque séjour, et de chaque hospitalisation

    left_join(sc %>% tbl('i2b2_patient') %>% select(patient_num, vital_status_cd, birth_date, death_date, sex_cd), by = "patient_num") %>%
    left_join(sc %>% tbl('i2b2_visit') %>% select(encounter_num, start_date, end_date), by = "encounter_num", suffix = c("_stay", "_visit")) %>%
    left_join(fa_de_novo) %>% 
    na.replace(fadenovo = 0) %>%
    semi_join(codages_fa, by = c('encounter_num', 'patient_num')) %>%
    semi_join(sc %>% tbl('i2b2_observation_lab'), by = c('encounter_num', 'patient_num')) %>%
    mutate(death_during_stay = ifelse(vital_status_cd == "V", 0, ifelse((death_date == end_date_stay), 1, 0))) %>%
    mutate(age_at_stay_entry = round((unix_timestamp(start_date_stay) - unix_timestamp(birth_date))/31536000, 0)) %>%
    mutate(death_during_visit = ifelse(vital_status_cd == "V", 0, ifelse((death_date == end_date_visit), 1, 0))) %>%
    mutate(duree_sejour = ceiling((unix_timestamp(end_date_stay) - unix_timestamp(start_date_stay))/86400)) %>%
    mutate(duree_visite = ceiling((unix_timestamp(end_date_visit) - unix_timestamp(start_date_visit))/86400)) %>%
    mutate(type_rea = ifelse(location_cd %in% liste_reas_med, 'med', 'chir')) %>%

## Plusieurs questionnements autour des séjours :

## Le filtre de durée de séjour > 2 jours peut sembler inquiétante vu qu c'est une condition de sélection de la cohorte.
## En réalité, il s'agit des séjours très courts associés à des séjours de plus de deux jours : exemple d'un patient initialement inscrit dans l'UF d'USC pour quelques heures 
## avant d'être passé dans l'UF de réanimation.

## Est-ce qu'un séjour de 3 jours en USC, puis, dans les suites d'une complication ou d'une aggravation, de 5 jours en réanimation, doivent compter comme un seul séjour ou 
## comme deux séjours ? C'est une vraie question à plusieurs égards
### Sur le plan de l'unité conceptuelle du séjour : est-ce plus pertinent de diviser ou d'unifier les gravités / mécanismes physiopathologiques / événements des deux séjours ?
### Sur le plan du choix fait de ne prendre que le premier séjour de chaque patient (choix discutable bien sur) en ignorant donc potentiellement un séjour d'USC ou de réanimation dans une même visite
### Sur le plan du data management, parce que fusionner deux séjours d'USC et de réanimation contigus lors de la même visite est une opération qui me semble difficile à réaliser (à voir si cela vaut 
### la peine d'investir le temps pour). Pour le moment, j'ai donc décidé d'enlever les UF de soins continus des UF de réanimation qui font partie du même service, pour ne pas avoir à gérer ce problème.
### La justification scientifique de cette supression est que nous ne prenons que les malades de réanimation.

    filter(duree_sejour >= 2 && duree_sejour < 365) %>%
    group_by(patient_num) %>%
    filter(start_date_stay == min(start_date_stay)) %>%
    ungroup()

Joining, by = c("encounter_num", "patient_num", "location_cd")


### Score CHA2DS2-VASc

In [23]:
## Score CHADS-VASC-2 par séjour de réanimation
## ATCD d'AVC : 2 points
## ATCD de pathologie vasculaire (AOMI, SCA) : 1 point
## ATCD d'HTA : 1 point
## ATCD de diabète : 1 point
## ATCD d'insuffisance cardiaque : 1 point
## Age compris entre 65 et 74 ans : 1 point
## Âge supérieur ou égal à 75 ans : 2 points
## Sexe féminin : 1 point

sejours_rea_48_heures <- sc %>% tbl('i2b2_observation_cim10') %>%
    select(encounter_num, tval_char, location_cd, concept_cd) %>%
    filter(location_cd %in% liste_reas) %>%
    mutate(concept_cd_abr = str_sub(concept_cd, 1, 9)) %>%
    filter(tval_char != "DP") %>%
    # Création de codes abrégés afin de rassembler sous une même catégorie les différents diagnostics détaillés
    filter(concept_cd_abr == "CIM10:I69" | concept_cd_abr == "CIM10:I25" | concept_cd_abr == "CIM10:I70" | concept_cd_abr == "CIM10:I10" | concept_cd_abr == "CIM10:E10" | concept_cd_abr == "CIM10:E11" | concept_cd_abr == "CIM10:I50") %>%
    # Nouvelle variable item_chads_vasc pour grouper plusieurs codes abrégés sous le même libellé
    mutate(item_chads_vasc = case_when (
        concept_cd_abr == "CIM10:I69" ~ "avc",
        concept_cd_abr == "CIM10:I25" | concept_cd_abr == "CIM10:I70" ~ "vasculaire",
        concept_cd_abr == "CIM10:I10" ~ "hta",
        concept_cd_abr == "CIM10:E10" | concept_cd_abr == "CIM10:E11" ~ "diabete",
        concept_cd_abr == "CIM10:I50" ~ "ins_card"
        )) %>%
    # Attribution du score de 1 à tous les codes, sauf celui d'antécédent d'AVC (I69) qui vaut 2
    mutate(point_chads_vasc = ifelse(item_chads_vasc == "avc", 2, 1)) %>%
    # Il faut enlever les codages détaillés...
    select(-concept_cd,-tval_char) %>%
    # pour pouvoir distinguer chaque ligne de codage abrégé...
    distinct() %>%
    # pour ensuite les grouper par visite hospitalière et par séjour en réanimation...
    group_by(encounter_num, location_cd) %>%
    # et sommer l'ensemble des points de chaque visite
    summarise(chads_vasc = sum(point_chads_vasc)) %>%
    right_join(sejours_rea_48_heures) %>%
    # Remplace les valeurs non attribuées (du fait du right_join qui crée des NaN à partir des patients n'ayant aucun codage sus-mentionné)
    na.replace(chads_vasc = 0) %>%
    # Ajoute le sexe et l'âge pour compléter le score
    mutate(chads_vasc = case_when(
        sex_cd == "W" ~ chads_vasc +1,
        sex_cd == "M" ~ chads_vasc
    )) %>%
    mutate(chads_vasc = case_when(
        age_at_stay_entry > 64 ~ chads_vasc +1,
        age_at_stay_entry > 74 ~ chads_vasc +2,
        age_at_stay_entry < 65 ~ chads_vasc
    ))

Joining, by = c("encounter_num", "location_cd")
Warning message:
“Missing values are always removed in SQL.
Use `SUM(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”
Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


## Tables des diagnostics CIM10
### Tables des complications
#### Complications hémorragiques

In [24]:
cim10_hemorragie <- sc %>% tbl('i2b2_observation_cim10') %>%
    select(encounter_num, patient_num, concept_cd, start_date, instance_num, tval_char, location_cd) %>%
    filter(location_cd %in% liste_reas) %>%

# Filtre pour retirer les admissions dont le DP est l'événement
# Pourrait se discuter de l'enlever mais il est difficile de faire la différence entre une hospitalisation pour un événement hémorragique dont la cause est le traitement anticoagulant ou non
    filter(tval_char != "DP") %>%
    

# Collect() est nécessaire pour créer un objet local sans quoi str_detect ne peut être effectuée
# Filtre pour ne sélectionner que les codes des événements hémorragiques
# Regex_left_join permet de joindre les tables hemorragie et i2b2_observation_cim10 pour ajouter la colonne 'type' de l'hémorragie, en se basant sur une expression régulière (pour que I617 soit reconnu comme un saignement intracrânien)
# Création d'une nouvelle variable avec les codes abrégés (une lettre et deux chiffres) pour regrouper les événements en grandes classes

    collect() %>%
    filter(str_detect(concept_cd, paste(hemorragie$code, collapse = "|"))) %>%
    regex_left_join(hemorragie, by = c(concept_cd = "code")) %>%
    mutate(concept_cd_abr = str_sub(concept_cd, 1, 9)) %>%
    mutate(type_rea = ifelse(location_cd %in% liste_reas_med, 'med', 'chir'))

In [25]:
cim10_hemorragie_wider <- cim10_hemorragie %>%
    pivot_wider(names_from = type, values_from = binaire_hemorragie, names_prefix = "hemorragie_", values_fill = 0) %>%
    select(-code, -instance_num, -concept_cd, -concept_cd_abr, -start_date, -tval_char, -type_rea) %>%
    group_by(encounter_num, patient_num, location_cd) %>%
    summarise(across(everything(), sum)) %>%
    mutate(across(starts_with("hemorragie"), .fns = ~ifelse(. > 0, 1, 0)))

`summarise()` has grouped output by 'encounter_num', 'patient_num'. You can
override using the `.groups` argument.


#### Complications ischémiques

In [26]:
cim10_ischemie <- sc %>% tbl('i2b2_observation_cim10') %>%
    select(encounter_num, patient_num, concept_cd, start_date, instance_num, tval_char, location_cd) %>%
    filter(location_cd %in% liste_reas) %>%

# Filtre pour retirer les admissions dont le DP est l'événement ischémique
# Pourrait se discuter de l'enlever mais il est difficile de faire la différence entre une hospitalisation pour un événement hémorragique dont la cause est le traitement anticoagulant ou non
    filter(tval_char != "DP") %>%

# Collect() est nécessaire pour créer un objet local sans quoi str_detect ne peut être effectuée
# Filtre pour ne sélectionner que les codes des événements ischémiques
# Regex_left_join permet de joindre les tables hemorragie et i2b2_observation_cim10 pour ajouter la colonne 'type' de l'hémorragie, en se basant sur une expression régulière (pour que I631 soit reconnues comme une ischémie cérébrale)
# Création d'une nouvelle variable avec les codes abrégés (une lettre et deux chiffres) pour regrouper les événements en grandes classes
    
    collect() %>%
    filter(str_detect(concept_cd, paste(ischemie$code, collapse = "|"))) %>%
    regex_left_join(ischemie, by = c(concept_cd = "code")) %>%
    mutate(concept_cd_abr = str_sub(concept_cd, 1, 9)) %>%
    mutate(type_rea = ifelse(location_cd %in% liste_reas_med, 'med', 'chir'))

In [27]:
cim10_ischemie_wider <- cim10_ischemie %>%
    pivot_wider(names_from = type, values_from = binaire_ischemie, names_prefix = "ischemie_", values_fill = 0) %>%
    select(-code, -instance_num, -concept_cd, -concept_cd_abr, -start_date, -tval_char, -type_rea) %>%
    group_by(encounter_num, patient_num, location_cd) %>%
    summarise(across(everything(), sum)) %>%
    mutate(across(starts_with("ischemie"), .fns = ~ifelse(. > 0, 1, 0)))

`summarise()` has grouped output by 'encounter_num', 'patient_num'. You can
override using the `.groups` argument.


### Etats de choc

In [28]:
cim10_choc <- sc %>% tbl('i2b2_observation_cim10') %>%
    select(encounter_num, patient_num, concept_cd, start_date, instance_num, location_cd) %>%
    filter(location_cd %in% liste_reas) %>%
    filter(concept_cd %in% !!etat_choc$code) %>%
    inner_join(sejours_rea_48_heures %>% select(encounter_num, patient_num, location_cd), by = c("encounter_num", "patient_num", "location_cd")) %>%
    collect() %>%
    regex_left_join(etat_choc, by = c(concept_cd = "code")) %>%
    select(-concept_cd, -start_date) %>%
    pivot_wider(names_from = type_choc, values_from = binaire_choc, names_prefix = "choc_", values_fill = 0) %>%
    select(-code, -instance_num) %>%
    group_by(encounter_num, patient_num, location_cd) %>%
    summarise(across(everything(), sum)) %>%
    mutate(across(starts_with("choc"), .fns = ~ifelse(. > 0, 1, 0)))

`summarise()` has grouped output by 'encounter_num', 'patient_num'. You can
override using the `.groups` argument.


### Antécédents

In [29]:
cim10_antecedents <- sc %>% tbl('i2b2_observation_cim10') %>%
    select(encounter_num, patient_num, concept_cd, start_date, instance_num, location_cd) %>%
    filter(location_cd %in% liste_reas) %>%
    collect() %>%
    filter(str_detect(concept_cd, paste(antecedents$code, collapse = "|"))) %>%
    inner_join(collect(sejours_rea_48_heures) %>% select(encounter_num, patient_num, location_cd), by = c("encounter_num", "patient_num", "location_cd")) %>%
    regex_left_join(antecedents, by = c(concept_cd = "code")) %>%
    select(-concept_cd, -start_date) %>%
    pivot_wider(names_from = type, values_from = binaire_atcd, names_prefix = "atcd_", values_fill = 0) %>%
    select(-code, -instance_num) %>%
    group_by(encounter_num, patient_num, location_cd) %>%
    summarise(across(everything(), sum)) %>%
    mutate(across(starts_with("atcd"), .fns = ~ifelse(. > 0, 1, 0)))

`summarise()` has grouped output by 'encounter_num', 'patient_num'. You can
override using the `.groups` argument.


### Cause d'admission

In [30]:
cim10_cat_admission <- sc %>% tbl("i2b2_observation_cim10") %>%
    filter(tval_char == "DP" && location_cd %in% liste_reas) %>%
    group_by(encounter_num) %>%
    filter(start_date == min(start_date)) %>%
    group_by(encounter_num) %>%
    filter(instance_num == min(instance_num)) %>%
    collect() %>%
    mutate(
        cat_admission_abr = str_sub(concept_cd, 7, 9),  # extraction des 3 premières lettres
        diag_cat = case_when(
           # 1. Pathologies neurochirurgicales / neurologiques aiguës
           cat_admission_abr %in% c("I60", "I61", "I62", "I63", "I64", "G40", "G93", "S06", "C71") ~ "Neuro",

           # 2. Pathologies traumatiques extra-crâniennes
           str_starts(cat_admission_abr, "S") & !cat_admission_abr %in% c("S06") ~ "Trauma_non_cranien",

           # 3. Pathologies cardio-vasculaires aiguës
           cat_admission_abr %in% c("Z95", "I21", "I22", "I24", "I50", "I48", "I01", "I31", "I51", "I46", "I71") | concept_cd == "CIM10:R570" ~ "Cardio",

           # 4. Pathologies thromboemboliques
           cat_admission_abr %in% c("I26", "I80", "I74") ~ "Thrombo_emb",

           # 5. Pathologies infectieuses sévères
           cat_admission_abr %in% c("A41", "J15", "J18", "N10", "G00", "G06", "I33") | concept_cd == "CIM10:R572" ~ "Sepsis",

           # 6. Pathologies hépatiques sévères
           cat_admission_abr %in% c("K70", "K72", "K74", "I85") ~ "Hepatique",

           # 7. Pathologies hématologiques / coagulopathies
           cat_admission_abr %in% c("D65", "D69", "C92", "C91", "D46", "D62", "K92") | concept_cd == "CIM10:R571" ~ "Hemato",

           # 8. Pathologies chirurgicales digestives
           cat_admission_abr %in% c("K65", "K85", "K83", "K63", "K35", "K38", "Z48", "K55") ~ "Post_op",

           # 9. Pathologies respiratoires isolées
           cat_admission_abr %in% c("J44", "J45", "J46", "J96", "J20", "J69", "J80", "U07", "J13") ~ "Respi",

           # 10. Intoxications / métaboliques / divers
           cat_admission_abr %in% c("T36", "T50", "E87", "E86", "E10", "E11", "F19", "N17") ~ "Metabo",

           TRUE ~ "Autre"
         )) %>%
    select(c(encounter_num, patient_num, cat_admission_abr, diag_cat))

## Tests d'anticoagulation
### Table initiale 

Tests d'anticoagulation par type d'anticoagulant et séjours de réanimation

In [31]:
anticoag_patients_sejours_rea <- sc %>% tbl('i2b2_observation_lab') %>%
    select(encounter_num, patient_num, concept_cd, instance_num, nval_num, quantity_num, start_date) %>%
    rename(instance_num_lab = instance_num) %>%
    rename(start_date_lab = start_date) %>%

### Décommenter pour modifier le type de tests à filtrer
    filter(concept_cd %in% lab_tous_tests_sans_tca) %>%

###    filter(concept_cd %in% lab_tous_tests) %>%

### Semi-join pour ne retenir que les visites avec au moins un séjour de réanimation
    semi_join(sejours_rea_48_heures, by = "encounter_num") %>%

### Inner Join pour grouper la table de labos et la table de séjours.
### La jointure se fait sur le numéro de patient et de visite, on ne peut donc pas avoir les examens de laboratoire qui 
### sont réalisés pendant les séjours de réanimations sur cette étape.

    inner_join(sejours_rea_48_heures, by = c("encounter_num", "patient_num")) %>%

### Filtres de dates : date de réalisation de l'examen > date d'admission en réanimation et < date de sortie de réanimation, 
### pour pouvoir n'obtenir que les examens réalisés pendant les séjours de réanimation.
    filter(unix_timestamp(start_date_lab) >= unix_timestamp(start_date_stay)) %>%
    filter(unix_timestamp(start_date_lab) <= unix_timestamp(end_date_stay)) %>%

### Création d'une variable catégorielle du type d'anticoagulation

    mutate(type_anticoag = case_when(
         concept_cd %in% lab_antiXa_hnf ~ 'hnf',
         concept_cd %in% lab_antiXa_hbpm ~ 'hbpm',
         concept_cd %in% lab_aod ~ 'aod',
         concept_cd %in% lab_tca_ratio ~ 'tca')
           )
### Affichage de la requête SQL si besoin   %>% show_query()

### Table statistique

Valeurs descriptives par séjour et par type d'anticoagulant

In [32]:
summary_anticoag_sejours_rea <- anticoag_patients_sejours_rea %>%
    group_by(instance_num, type_anticoag) %>%
    summarise (nb_tests = n(), 
               moyenne = round(mean(nval_num),2),
               mediane = round(percentile(nval_num,0.5),2),
               min = min(nval_num), 
               max = max(nval_num), 
               sd = round(sd(nval_num),2), 
               q25=round(percentile(nval_num,0.25),2), 
               q75=round(percentile(nval_num,0.75),2))

### Table aggrégée

Reprise des valeurs statistiques par séjour et type d'anticoagulant

Ajout des valeurs suivantes : 
 - Nombre total de tests par séjour
 - Ratio nombre de tests / durée de séjour

In [33]:
donnees_aggregees_anticoag <- anticoag_patients_sejours_rea %>% 
    group_by(instance_num) %>%

    ## Compte du nombre total de tests par sejours
    summarise(nb_total_tests_par_sejour = n()) %>%

    ## Right_join pour reprendre les données de la table anticoag_patients_sejours_rea
    right_join(anticoag_patients_sejours_rea) %>%

    ## pour pouvoir calculer le ratio entre nombre total de tests et DMS
    mutate(ratio_tests_duree_sejour = round((nb_total_tests_par_sejour / duree_sejour),2)) %>%

    ## Group et Distinct retirent les répétitions de chaque ligne issues de la fusion entre summarise et right_join
    group_by(instance_num) %>%
    ## Sélection des variables d'intérêt
    select(instance_num, encounter_num, nb_total_tests_par_sejour, duree_sejour, ratio_tests_duree_sejour, type_anticoag) %>%
    distinct() %>%

    ## Left_join avec la table qui récapitule les valeurs statistiques principales par séjour et par type de test d'anticoagulation
    left_join(summary_anticoag_sejours_rea) %>%

    ## Retire instance_num en double issu du left_join
    select(-instance_num.x)

Joining, by = "instance_num"
Joining, by = c("instance_num", "type_anticoag")


### Table filtrée

Choix du filtre pour déterminer trois niveaux d'anticoagulation : 

 - Efficace
 - Imparfait
 - Inefficace

In [34]:
donnees_aggregees_filtrees_anticoag <- donnees_aggregees_anticoag %>%

# Conditions de filtrage des séjours de rénanimation où on considère que on a voulu anticoaguler le patient.
# La question épineuse est de savoir quels critères ou quelle combinaison de critères prendre.

# Ci dessous : le négatif (!) d'un nombre de tests inférieurs à trois ET d'une valeur maximale <= à 0.20

#  mutate(volonte_anticoag = if_else((type_anticoag=='hnf' | type_anticoag=='hbpm') && (!(nb_tests < 3 && max <= 0.20)), 1, 0)) %>%

## 3604 séjours au 15/03

# Ci dessous : le négatif (!) d'un nombre de tests inférieurs à trois ET d'une valeur maximale <= à 0.10

#  mutate(volonte_anticoag = if_else((type_anticoag=='hnf' | type_anticoag=='hbpm') && (!(nb_tests < 3 && max <= 0.10)), 1, 0)) %>%

## 3929 séjours au 15/03

## Autre proposition de filtre, à voir lequel est le meilleur

   mutate(volonte_anticoag = if_else((type_anticoag=='hnf' | type_anticoag=='hbpm') && (!(ratio_tests_duree_sejour < 0.30 | (max <= 0.10 && nb_tests < 3))), 1, 0)) %>%

## 3344 séjours au 15/03

## Code à décommenter pour recalculer le nombre de séjours de chaque filtre :
## donnees_aggregees_filtrees_anticoag %>%
##    filter(volonte_anticoag == 1) %>%
##    count() %>%
##    count()

## Création d'une variable catégorielle dichotomisant l'échec d'anticoagulation (q75 < 0,10 pour HNF, < 0,3 pour HBPM), la réussite d'anticoagulation (q25 > 0,3 pour HNF, > 0,5 pour HBPM)
### et une anticoagulation imparfaite (tout le reste). Une question est : faut-il faire plus de catégories ?

    mutate(efficacite_anticoag = case_when(
        volonte_anticoag == 1 && type_anticoag == "hnf" && q25 > 0.3 ~ "efficace",
        volonte_anticoag == 1 && type_anticoag == "hnf" && q75 < 0.11  ~ "inefficace",
        volonte_anticoag == 1 && type_anticoag == "hbpm" && q25 > 0.5 ~ "efficace",
        volonte_anticoag == 1 && type_anticoag== "hbpm" && q75 < 0.31  ~ "inefficace",
        volonte_anticoag == 0 ~ NA,
        TRUE ~ "imparfait")) %>%

## Création d'une variable binaire de surdosage (initialement je pensais mettre surdosage dans un type d'anticoagulation, mais il semble logique de considérer des cas
## d'anticoagulation inefficace ou imparfaite avec un surdosage, surtout si on veut potentiellement examiner l'association d'un surdosage avec un événement hémorragique)
           
    mutate(surdosage = ifelse(type_anticoag == "hnf" && max > 0.7 | type_anticoag== "hbpm" && max > 1.2, 1,0))

In [35]:
donnees_aggregees_filtrees_anticoag_wider <- donnees_aggregees_filtrees_anticoag %>%
    select(-c(encounter_num,duree_sejour)) %>%
    collect() %>%
    pivot_wider(names_from = type_anticoag, values_from = c(nb_tests:surdosage), names_glue = "{type_anticoag}_{.value}") %>%
    mutate(volonte_anticoag = case_when (
        hbpm_volonte_anticoag == 1 | hnf_volonte_anticoag == 1 ~ 1,
        TRUE ~ 0))

Warning message:
“Missing values are always removed in SQL.
Use `mean(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”
Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”
Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”


## Autres tests de laboratoire

### Table initiale

In [36]:
# Table des examens de laboratoire pertinents, en dehors des examens de laboratoire d'hémostase
# Pour l'instant : hémoglobine, urée, fibrinogène, plaquettes, DDimères

labos_patients_sejours_rea <- sc %>% tbl('i2b2_observation_lab') %>%
    rename(instance_num_lab = instance_num) %>%
    rename(start_date_lab = start_date) %>%
    filter(concept_cd %in% lab_tous_autres |
           
## Problème, un certain nombre de séjours n'ont pas d'Hb NFS, et il y a un certain nombre de codes 
## dits LAB:INCONNU qui sont des Hb.
           
           concept_cd == "LAB:INCONNU" && (units_cd == "g/dl" | units_cd == "g/dL") && 
           (confidence_num == "17" | confidence_num == "17.5" | confidence_num == "16")) %>%
    select(start_date_lab, instance_num_lab, nval_num, encounter_num, patient_num, quantity_num, units_cd, confidence_num) %>%

### Création de catégories d'examens de laboratoire en groupant les différents examens de laboratoire dosant les même paramètres

    mutate(concept_cd_cat = case_when (
        concept_cd == "LAB:INCONNU" && (units_cd == "g/dl" | units_cd == "g/dL") && (confidence_num == "17" | confidence_num == "17,5" | confidence_num == "16") ~ "hemoglobine",
        concept_cd %in% lab_hb ~ "hemoglobine",
        concept_cd %in% lab_uree ~ "uree",
        concept_cd %in% lab_fib ~ "fibrinogene",
        concept_cd %in% lab_plq ~ "plaquettes",
        concept_cd %in% lab_ddim ~ "ddimeres",
        concept_cd %in% lab_creat ~ "creat"
    )) %>%

    semi_join(sejours_rea_48_heures, by = c("encounter_num")) %>%

### Inner Join pour grouper la table de labos et la table de séjours.
### La jointure se fait sur le numéro de patient et de visite, on ne peut donc pas avoir les examens de laboratoire qui 
### sont réalisés pendant les séjours de réanimation sur cette étape (la variable instance_num ne désigne pas le même objet dans les deux tables, malheureusement)

    inner_join(sejours_rea_48_heures, by = c("encounter_num", "patient_num")) %>%

### Filtres de dates : date de réalisation de l'examen > date d'admission en réanimation et < date de sortie de réanimation, 
### pour pouvoir n'obtenir que les examens réalisés pendant les séjours de réanimation.

    filter(unix_timestamp(start_date_lab) >= unix_timestamp(start_date_stay)) %>%
    filter(unix_timestamp(start_date_lab) <= unix_timestamp(end_date_stay)) %>%
    select(-instance_num_lab)

### Table statistique aggrégée

In [37]:
# Création d'une table de labo groupée par type d'examen et par séjour pour en obtenir les valeurs statistiques suivantes : 
# moyenne, mediane, minimum, maximum, écart-type, interquartiles

donnees_aggregees_labos <- labos_patients_sejours_rea %>%
    group_by(instance_num, concept_cd_cat) %>%
    summarise(nb_tests = n(),
               moyenne = round(mean(nval_num),2),
               mediane = round(percentile(nval_num,0.5),2),
               min = min(nval_num), 
               max = max(nval_num),
               sd = round(sd(nval_num),2), 
               q25=round(percentile(nval_num,0.25),2), 
               q75=round(percentile(nval_num,0.75),2))

In [38]:
donnees_aggregees_labos_wider <- donnees_aggregees_labos %>%
    collect() %>%
    pivot_wider(names_from = concept_cd_cat, values_from = c(nb_tests:q75), names_glue = "{concept_cd_cat}_{.value}") %>%
    distinct(instance_num, .keep_all = TRUE)

## Actes CCAM
### Table initiale

In [39]:
actes_ccam <- sc %>% tbl('i2b2_observation_ccam') %>%
## Filtre des UFR de réanimation
    filter(location_cd %in% liste_reas) %>%
## Sélection des variables d'interêt
    select(patient_num, encounter_num, start_date, location_cd, concept_cd, start_date) %>%
## Renommage de la date de codage de l'acte (vérifier avec l'EDS pourquoi 22 et 23h uniquement ?) pour ne pas doublonner
    rename(start_date_ccam = start_date) %>%

## Jointure avec la table des séjours en sélectionnant les variables d'intérêt
    inner_join(sejours_rea_48_heures %>% select(encounter_num, instance_num, location_cd, start_date_stay, end_date_stay), by=c('encounter_num', 'location_cd')) %>%

## Filtrage en fonction des dates (certains codages d'une UFR donnée sont parfois entrés avant le début du séjour, à vérifier avec l'EDS)

    filter(unix_timestamp(start_date_ccam) >= unix_timestamp(start_date_stay)) %>%
    filter(unix_timestamp(start_date_ccam) <= unix_timestamp(end_date_stay)) %>%
    distinct() %>%
    collect()

### Ventilation mécanique

In [40]:
jours_vm <- actes_ccam %>%
    filter(concept_cd %in% ccam_vm) %>%
    group_by(instance_num, start_date_ccam, concept_cd) %>%
    summarise(n = n()) %>%
    group_by(instance_num) %>%
    summarise(n = n()) %>%
    rename(jours_vm = n)

`summarise()` has grouped output by 'instance_num', 'start_date_ccam'. You can
override using the `.groups` argument.


### Vasopresseurs

In [41]:
jours_vasopresseurs <- actes_ccam %>%
    filter(concept_cd %in% ccam_vasopresseur) %>%
    group_by(instance_num, start_date_ccam, concept_cd) %>%
    summarise(n = n()) %>%
    group_by(instance_num) %>%
    summarise(n = n()) %>%
    rename(jours_vasopresseurs = n)

`summarise()` has grouped output by 'instance_num', 'start_date_ccam'. You can
override using the `.groups` argument.


### Dialyse

In [42]:
jours_eer <- actes_ccam %>%
    filter(concept_cd %in% ccam_eer) %>%
    group_by(instance_num, start_date_ccam, concept_cd) %>%
    summarise(n = n()) %>%
    group_by(instance_num) %>%
    summarise(n = n()) %>%
    rename(jours_eer = n)

`summarise()` has grouped output by 'instance_num', 'start_date_ccam'. You can
override using the `.groups` argument.


### Coagulopathie

In [43]:
jours_coagulopathie <- actes_ccam %>%
    filter(concept_cd %in% ccam_coagulopathie) %>%
    group_by(instance_num, start_date_ccam, concept_cd) %>%
    summarise(n = n()) %>%
    group_by(instance_num) %>%
    summarise(n = n()) %>%
    rename(jours_coagulopathie = n)

`summarise()` has grouped output by 'instance_num', 'start_date_ccam'. You can
override using the `.groups` argument.


### Transfusion

In [44]:
jours_transfusion <- actes_ccam %>%
    filter(concept_cd %in% ccam_transfusion) %>%
    group_by(instance_num, start_date_ccam, concept_cd) %>%
    summarise(n = n()) %>%
    group_by(instance_num) %>%
    summarise(n = n()) %>%
    rename(jours_transfusion = n)

`summarise()` has grouped output by 'instance_num', 'start_date_ccam'. You can
override using the `.groups` argument.


## Séjours
  - intégration des variables des autres tables
  - élimination des diagnostics et des actes interférant avec l'exposition à l'anticoagulation

In [45]:
sejours_rea_48_heures <- sejours_rea_48_heures %>%
    collect() %>%
    left_join(jours_vm) %>%
    left_join(jours_vasopresseurs) %>%
    left_join(jours_eer) %>%
    left_join(jours_transfusion) %>%
    left_join(jours_coagulopathie) %>%
    mutate(vm = ifelse(is.na(jours_vm), 0, 1)) %>%
    mutate(vasopresseurs = ifelse(is.na(jours_vasopresseurs), 0, 1)) %>%
    mutate(eer = ifelse(is.na(jours_eer), 0, 1)) %>%
    mutate(transfusion = ifelse(is.na(jours_transfusion), 0, 1)) %>%
    mutate(coagulopathie = ifelse(is.na(jours_coagulopathie), 0, 1)) %>%
    left_join(donnees_aggregees_filtrees_anticoag_wider, by="instance_num") %>%
    replace_na(list(volonte_anticoag = 0)) %>%
    left_join(donnees_aggregees_labos_wider, by="instance_num") %>%
    left_join(cim10_choc) %>%
    left_join(cim10_ischemie_wider) %>%
    left_join(cim10_hemorragie_wider) %>%
    left_join(cim10_antecedents) %>%
    left_join(cim10_cat_admission) %>%
    mutate(across(starts_with(c("choc","ischemie","hemorragie","b_", "atcd")), .fns = ~ifelse(is.na(.), 0, .))) %>%
    anti_join(cim10_biais_hemorragie, by=c("encounter_num", "patient_num")) %>%
    anti_join(cim10_biais_thrombose, by=c("encounter_num", "patient_num")) %>%
    anti_join(actes_ccam %>% filter(concept_cd %in% ccam_ecmo), by=c("encounter_num", "patient_num")) 

Joining with `by = join_by(instance_num)`
Joining with `by = join_by(instance_num)`
Joining with `by = join_by(instance_num)`
Joining with `by = join_by(instance_num)`
Joining with `by = join_by(instance_num)`
Joining with `by = join_by(encounter_num, location_cd, patient_num)`
Joining with `by = join_by(encounter_num, location_cd, patient_num)`
Joining with `by = join_by(encounter_num, location_cd, patient_num)`
Joining with `by = join_by(encounter_num, location_cd, patient_num)`
Joining with `by = join_by(encounter_num, patient_num)`


# Ecriture des tables

In [46]:
actes_ccam <- copy_to(sc, actes_ccam, overwrite = TRUE)

In [47]:
spark_write_parquet(actes_ccam, "actes_ccam", mode="overwrite")

In [48]:
sejours_rea_48_heures <- copy_to(sc, sejours_rea_48_heures, overwrite = TRUE)

In [49]:
spark_write_parquet(sejours_rea_48_heures, 'sejours_rea_48_heures', mode='overwrite')

In [50]:
spark_write_parquet(anticoag_patients_sejours_rea, "anticoag_patients_sejours_rea", mode="overwrite")

In [51]:
spark_write_parquet(donnees_aggregees_anticoag, "donnees_aggregees_anticoag", mode="overwrite")

In [52]:
spark_write_parquet(donnees_aggregees_filtrees_anticoag, "donnees_aggregees_filtrees_anticoag", mode="overwrite")

In [53]:
spark_write_parquet(donnees_aggregees_labos, "donnees_aggregees_labos", mode="overwrite")

In [54]:
spark_write_parquet(labos_patients_sejours_rea, "labos_patients_sejours_rea", mode="overwrite")

# Garbage collector

In [55]:
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2809638,150.1,5022080,268.3,5022080,268.3
Vcells,5593946,42.7,21618226,165.0,22594801,172.4
